<a href="https://www.kaggle.com/code/yugamjayant/d05-transfer-learning-model-02?scriptVersionId=155623831" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/keras_metadata.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.data-00000-of-00001
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.index
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.data-00000-of-00001
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv
/kaggle/input/ll

In [2]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
import random
import tqdm
from tqdm import tqdm
import copy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_essays_01_source = pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv')

#Slicing the dataset with different labels
train_essays_01_source_label_1 = train_essays_01_source[train_essays_01_source['label'] == 1].reset_index(drop = True)  
train_essays_01_source_label_0 = train_essays_01_source[train_essays_01_source['label'] == 0].reset_index(drop = True)

#Set number of data points required from each label
#Creating new data set with data sampled from each labels
label_1_count = 1638 #1638 data points available
label_0_count = 13712 #13712 data points available

essay_dataset_raw = pd.concat([train_essays_01_source_label_1.sample(n= label_1_count, random_state = 36, axis = 0), 
                         train_essays_01_source_label_0.sample(n= label_0_count, random_state = 36, axis = 0)], 
                        axis = 0, sort = True).sample(frac = 1)

In [4]:
#Bert encoder

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2",trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [5]:
encoder_model = tf.keras.Model(inputs=text_input, outputs=sequence_output)

In [6]:
%%time
batch_size = 100
num_batches = len(essay_dataset_raw)//batch_size

essay_dataset_raw_batches = np.array_split(essay_dataset_raw.iloc[:,1], num_batches)
essay_dataset_raw_batch_list = []
i = 0

for batch in tqdm(essay_dataset_raw_batches):
    i+=1
    encoded_essay_batch = encoder_model(batch)
    essay_dataset_raw_batch_list.append(encoded_essay_batch)
    
    if i%100 == 0:
        encoded_essay = np.concatenate(essay_dataset_raw_batch_list, axis = 0)
        del essay_dataset_raw_batch_list
        essay_dataset_raw_batch_list = [encoded_essay]
    
    
encoded_essay = tf.convert_to_tensor(np.concatenate(essay_dataset_raw_batch_list, axis = 0))

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 153/153 [02:09<00:00,  1.18it/s]


CPU times: user 2min 30s, sys: 32.2 s, total: 3min 2s
Wall time: 2min 21s


In [7]:
del essay_dataset_raw_batches
del essay_dataset_raw_batch_list

In [8]:
#Train:CV:Test :: i:j:k 
i, j, k = 4, 3, 1

fraction_size = len(encoded_essay)//(i+j+k)
encoded_essay_train, encoded_essay_CV, encoded_essay_test = tf.split(encoded_essay, [fraction_size*i, fraction_size*j, len(encoded_essay)-fraction_size*(i+j)], 0)

label_train, label_CV, label_test = tf.split(essay_dataset_raw['label'], [fraction_size*i, fraction_size*j, len(essay_dataset_raw)-fraction_size*(i+j)], 0)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

# Import your input embeddings
input_embeddings = tf.keras.Input(shape=(128, 768, ))

# Apply pooling right after the input layer
pooled_input = layers.GlobalMaxPooling1D()(input_embeddings)

# Reshape pooled input to add singleton dimension
query = tf.expand_dims(pooled_input, axis=1)

# Multi-Head Attention layer with pooling layer output as key and input embedding vectors as values
num_heads = 12
key_dim = 64  # Adjust based on experimentation
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(query, input_embeddings)

# Global average pooling
global_pool = layers.GlobalAveragePooling1D()(attention_output)

# Dropout to prevent overfitting
dropout = layers.Dropout(0.2)(global_pool)

# Dense output layer with 1 unit and sigmoid activation (for binary classification)
output = layers.Dense(1, activation="sigmoid")(dropout)

# Create the model
model = tf.keras.Model(inputs=input_embeddings, outputs=output)


In [10]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy', 'AUC']
)

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 768)]           0         []                            
                                                                                                  
 global_max_pooling1d (Glob  (None, 768)                  0         ['input_2[0][0]']             
 alMaxPooling1D)                                                                                  
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 768)               0         ['global_max_pooling1d[0][0]']
 )                                                                                                
                                                                                            

In [12]:
h = model.fit(
   encoded_essay_train, label_train,
   validation_data = (encoded_essay_CV, label_CV),
   epochs = 20,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', patience = 1, mode = 'max', 
                                      restore_best_weights = True, min_delta = 0.01, start_from_epoch = 2)
   ]
)

Epoch 1/20
240/240 [==============================] - 9s 26ms/step - loss: 0.0992 - accuracy: 0.9690 - auc: 0.9735 - val_loss: 0.0855 - val_accuracy: 0.9666 - val_auc: 0.9978
Epoch 2/20
240/240 [==============================] - 4s 17ms/step - loss: 0.0342 - accuracy: 0.9898 - auc: 0.9945 - val_loss: 0.0298 - val_accuracy: 0.9892 - val_auc: 0.9983
Epoch 3/20
240/240 [==============================] - 4s 17ms/step - loss: 0.0309 - accuracy: 0.9902 - auc: 0.9955 - val_loss: 0.0762 - val_accuracy: 0.9708 - val_auc: 0.9910
Epoch 4/20
240/240 [==============================] - 4s 17ms/step - loss: 0.0260 - accuracy: 0.9930 - auc: 0.9953 - val_loss: 0.0272 - val_accuracy: 0.9901 - val_auc: 0.9978


In [13]:
# _ = model.evaluate(encoded_essay_test, label_test)

In [14]:
test_data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [15]:
%%time
# encoded_test_data = encoder_model(train_essays_01_source.iloc[:10000, 0])
# test_set_predictions = model.predict(encoded_test_data)


if len(test_data)<100:
    encoded_test_data = encoder_model(test_data.iloc[:, 2])
    test_set_predictions = model.predict(encoded_test_data)

else:
    batch_size = 100
    num_batches = len(test_data)//batch_size

    test_data_batches = np.array_split(test_data, num_batches)
    test_set_predictions_batch_list = []

    for batch in tqdm(test_data_batches):
        encoded_test_data_batch = encoder_model(batch.iloc[:, 2])
        test_set_predictions_batch = model.predict(encoded_test_data_batch)
        test_set_predictions_batch_list.append(test_set_predictions_batch)

    test_set_predictions = np.concatenate(test_set_predictions_batch_list, axis = 0)

1/1 [==============================] - 0s 107ms/step
CPU times: user 170 ms, sys: 25.2 ms, total: 195 ms
Wall time: 181 ms


In [16]:
test_data['genarated'] = np.round(test_set_predictions.flatten().tolist(), decimals = 4)
submission = test_data[['id', 'genarated']]
submission.to_csv('/kaggle/working/submission.csv', index = False, header = submission.columns.tolist())